# Regression Adjustment and CUPED

In this example, we will use Jonathan Roth's DGP with heterogenous effects. You are a data scientist at Udemy looking at the effects of taking a professional development $(D)$ certificate on earnings $(Y)$. You randomly assign a sample of individuals to get the certificate or not. Let $Z$ indicate how many online courses a person has taken in the past and $Y_{t-1}$ be their earnings last year.

Suppose that taking online courses causes lower earnings $Y(0)$ in jobs that don't require any certificates, but higher earnings $Y(1)$ in jobs that do require certificates. 


In [223]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

rng = np.random.default_rng(42)

In [224]:
# Sample size
n = 500

# Number of online courses
Z = rng.normal(20, 10, size=n)
Z = np.where(Z < 0, 0, Z)  # truncate Z to be non-negative

# Earnings before experiment
Ypre = rng.normal(60000, 3000, size=n)

# Potential earning
Y0 = -500*Z + Ypre + rng.normal(5000, 1000, size=n)
Y1 = 500*Z + 1.05*Ypre + rng.normal(5000, 1000, size=n)

# Random treatment and observed earnings
D = rng.binomial(1, .2, size=n)  # only 20% get treated
Y = Y1 * D + Y0 * (1 - D)

# Available data
data = pd.DataFrame({'Y': Y, 'D': D, 'Z': Z, 'Ypre': Ypre}).round(0).astype(int)
data

,Y,D,Z,Ypre
0,57509,0,23,64092
1,62156,0,10,62686
2,48675,0,28,57842
3,46424,0,29,55492
4,55865,0,0,51106
...,...,...,...,...
495,50276,0,29,61058
496,66780,0,2,62300
497,55891,0,17,60364
498,66058,0,0,60392


In [225]:
data.describe().round(0).astype(int)

,Y,D,Z,Ypre
count,500,500,500,500
mean,59313,0,20,59866
std,10476,0,10,3055
min,37850,0,0,49055
25%,52452,0,13,57825
50%,56796,0,20,60026
75%,63448,0,26,61788
max,92875,1,49,69537


## Regression Adjustment

In [226]:
# Classical 2-sample approach, no adjustment (CL)
CL = smf.ols("np.log(Y) ~ D", data=data).fit(cov_type='HC1')
print(CL.summary())

                            OLS Regression Results                            
Dep. Variable:              np.log(Y)   R-squared:                       0.634
Model:                            OLS   Adj. R-squared:                  0.634
Method:                 Least Squares   F-statistic:                     1364.
Date:                Wed, 04 Sep 2024   Prob (F-statistic):          1.00e-144
Time:                        14:17:49   Log-Likelihood:                 431.82
No. Observations:                 500   AIC:                            -859.6
Df Residuals:                     498   BIC:                            -851.2
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.9100      0.005   2027.965      0.0

In [227]:
# Classical linear regression adjustment (CRA)
CRA = smf.ols("np.log(Y) ~ D + Z + np.log(Ypre)", data=data).fit(cov_type='HC1')
print(CRA.summary())


                            OLS Regression Results                            
Dep. Variable:              np.log(Y)   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     564.5
Date:                Wed, 04 Sep 2024   Prob (F-statistic):          1.83e-159
Time:                        14:17:49   Log-Likelihood:                 702.82
No. Observations:                 500   AIC:                            -1398.
Df Residuals:                     496   BIC:                            -1381.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.7079      0.566     -1.250   

In [228]:
# Demean Z and Ypre
data['Z_dm'] = data['Z'] - data['Z'].mean()
data['Ypre_dm'] = np.log(data['Ypre']) - np.log(data['Ypre']).mean()

# Interactive regression adjusment (IRA)
IRA = smf.ols("np.log(Y) ~ D + Z_dm + Z_dm*D + Ypre_dm + Ypre_dm*D", data=data).fit(cov_type='HC1')
print(IRA.summary())

                            OLS Regression Results                            
Dep. Variable:              np.log(Y)   R-squared:                       0.986
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                 1.046e+04
Date:                Wed, 04 Sep 2024   Prob (F-statistic):               0.00
Time:                        14:17:49   Log-Likelihood:                 1252.4
No. Observations:                 500   AIC:                            -2493.
Df Residuals:                     494   BIC:                            -2468.
Df Model:                           5                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.9075      0.001   1.03e+04      0.0

In [229]:
print('CL se:', CL.bse['D'].round(5))
print('CRA se:', CRA.bse['D'].round(5))
print('IRA se:', IRA.bse['D'].round(5))

CL se: 0.0092
CRA se: 0.01191
IRA se: 0.00168


Observe that CRA delivers estimates that are less efficient than CL (pointed out by Freedman), whereas IRA delivers estimates that are more efficient (pointed out by Lin). In order for CRA to be more efficient than CL, we need the linear model to be a correct model of the conditional expectation function of Y given D and X, which is not the case here.

## CUPED: Controlled-Experiment using Pre-Experiment Data

This is a very popular technique in business settings to increase the power of RCTs.

For a recent perspective on CUPED, see 
- [A New Look at CUPED in 2023](https://arxiv.org/pdf/2312.02935)
- [Powering Experiments with CUPED](https://towardsdatascience.com/powering-experiments-with-cuped-and-double-machine-learning-34dc2f3d3284)
- [Understanding CUPED](https://matteocourthoud.github.io/post/cuped/).

Steps to implement CUPED:
1. Regress $Y$ on $X \equiv [Z, Y_{t-1}]$ and obtain the residuals $\hat{Y}_{\text{cuped}} = Y - \hat{\beta}X$.
2. Regress $\hat{Y}_{\text{cuped}}$ on $D$ and obtain the treatment effect

In [230]:
# Compute residuals
data['Y_tilde'] = smf.ols("np.log(Y) ~ Z_dm + Ypre_dm", data=data).fit().resid
cuped = smf.ols("Y_tilde ~ D", data=data).fit(cov_type='HC1')
print(cuped.summary())

                            OLS Regression Results                            
Dep. Variable:                Y_tilde   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     856.9
Date:                Wed, 04 Sep 2024   Prob (F-statistic):          2.64e-110
Time:                        14:17:50   Log-Likelihood:                 692.65
No. Observations:                 500   AIC:                            -1381.
Df Residuals:                     498   BIC:                            -1373.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0679      0.002    -37.743      0.0

In [231]:
print("CUPED se:", cuped.bse["D"].round(5))

CUPED se: 0.01196
